# Stochastic Simulations Part 1 - Genetic Toggle Switch

This notebook contains all of the figures generated in Chapter 2. All figures were generated as interactive visualizations using the bokeh library.


The notebook contains sections from the 'Design Principles of Genetic Circuits' course at Caltech, Spring term, 2019 (http://be150.caltech.edu/2019/). In particular, the choice of the Python packages was determined by the course. Some of the templates provided as part of the course were used to generate some of the figures. 

In [3]:
import multiprocessing
import tqdm

import numpy as np
import scipy.stats as st
import numba

import biocircuits

# Plotting modules
import bokeh.io
import bokeh.plotting

from bokeh.io import export_png

bokeh.io.output_notebook()

# Line profiler (can install with conda install line_profiler)
%load_ext line_profiler

Loading BokehJS ...

## The stochastic model for the toggle_switch

For the Gillespie simulations we need to specify the propensities and updates, along with an initial population. 

The model takes the follwowing form:
We explicitly consider both mRNA and protein. A repressor binds the operator with chemical rate constant $k_r$, and an operator may have zero, one, or two repressors bound. The unbinding rate of a repressor when one is bound is $k_{u,1}$ and that of a repressor when two are bound if $k_{u,2}$, with $k_{u,2} < k_{u,1}$ to capture cooperativity. If no repressors are bound to a promoter region transciption happens at rate constant $k_{mu}$. If one or two repressors are bound, basal transcription can occur, and transciption happens at rate constant $k_{mo}$.

This is the table of the populations we are keeping track of:

|index|description | variable|
|:----:|:------ |:------: |
|0|gene 1 mRNA copy number | `m1`|
|1|unbound gene 1 protein copy number | `p1`|
|2|gene 2 mRNA copy number | `m2`|
|3|unbound gene 2 protein copy number | `p2`|
|4|bound gene 2 protein copy number (Number of repressors bound to promoter of gene 1)| `n2`|
|5|bound gene 1 protein copy number (Number of repressors bound to promoter of gene 2)| `n1`|

Note that we labeled each species with an index, which corresponds to its position in the array of populations in the simulation.

Next, we can set up a table of updates and propensities for the moves we allow in the Gillespie simulation. We also assign an index to each entry here, as this helps keep track of everything.

|index|description | update | propensity|
|:----|:------ |:------ | :----:|
|0|transcription of gene 1 mRNA| `m1 ⟶ m1 + 1`| `kmu*(n2 == 0) + kmo*(n2 > 0)`|
|1|transcription of gene 2 mRNA| `m2 ⟶ m2 + 1`| `kmu*(n1 == 0) + kmo*(n1 > 0)`|
|2|translation of gene 1 protein| `p1 ⟶ p1 + 1`| `kp * m1`|
|3|translation of gene 2 protein| `p2 ⟶ p2 + 1`| `kp * m2`|
|4|degradation of gene 1 mRNA| `m1 ⟶ m1 - 1`| `gamma_m * m1`|
|5|degradation of gene 2 mRNA| `m2 ⟶ m2 - 1`| `gamma_m * m2`|
|6|degradation of unbound gene 1 protein| `p1 ⟶ p1 - 1` | `gamma_p * p1`|
|7|degradation of unbound gene 2 protein| `p2 ⟶ p2 - 1` | `gamma_p * p2`|
|8|degradation of bound gene 1 protein| `n1 ⟶ n1 - 1` | `gamma_p * n1`|
|9|degradation of bound gene 2 protein| `n2 ⟶ n2 - 1` | `gamma_p * n2`|
|10|binding of protein to gene 1 operator| `n2 ⟶ n2 + 1`, `p2 ⟶ p2 - 1`| `kr * p2 * (n2 < 2)`|
|11|binding of protein to gene 2 operator| `n1 ⟶ n1 + 1`, `p1 ⟶ p1 - 1`| `kr * p1 * (n1 < 2)`|
|12|unbinding of protein to gene 1 operator| `n2 ⟶ n2 - 1`, `p2 ⟶ p2 + 1`| `ku1*(n2 == 1) + 2*ku2*(n2 == 2)`|
|13|unbinding of protein to gene 2 operator| `n1 ⟶ n1 - 1`, `p1 ⟶ p1 + 1`| `ku1*(n1 == 1) + 2*ku2*(n1 == 2)`|

Finally, we have parameters that were introduced in the propensities, so we should have a table defining them.

|parameter| value | units |
|:----|:------ |:------: |
|`kmu`| 0.5 | 1/sec|
|`kmo`|0.00001  | 1/sec|
|`kp`| 0.167 | 1/molec-sec|
|`gamma_m`| 0.05776 | 1/sec|
|`gamma_p`| 0.00155 | 1/sec|
| `kr` | 1.0 | 1/molec-sec|
| `ku1` | 5.0 | 1/sec|
| `ku2` | 0.05 | 1/sec|

In [4]:
@numba.njit
def toggle_switch_propensity(propensities, population, t, 
                             kmu,
                             kmo,
                             kp,
                             gamma_m,
                             gamma_p,
                             kr,
                             ku1,
                             ku2):
    m1, p1, m2, p2, n2, n1 = population
    
    propensities[ 0] = kmu if n2 == 0 else kmo
    propensities[ 1] = kmu if n1 == 0 else kmo
    propensities[ 2] = kp * m1                
    propensities[ 3] = kp * m2                             
    propensities[ 4] = gamma_m * m1           
    propensities[ 5] = gamma_m * m2                   
    propensities[ 6] = gamma_p * p1           
    propensities[ 7] = gamma_p * p2                   
    propensities[ 8] = gamma_p * n1          
    propensities[ 9] = gamma_p * n2                  
    propensities[10] = kr * p2 * (n2 < 2)    
    propensities[11] = kr * p1 * (n1 < 2)     
    propensities[12] = ku1*(n2==1) + 2*ku2*(n2==2)
    propensities[13] = ku1*(n1==1) + 2*ku2*(n1==2)

In [5]:
toggle_switch_update = np.array([
    # 0   1   2   3   4   5
    #m1  p1  m2  p2  n2  n1
    [ 1,  0,  0,  0,  0,  0], # 0
    [ 0,  0,  1,  0,  0,  0], # 1
    [ 0,  1,  0,  0,  0,  0], # 2
    [ 0,  0,  0,  1,  0,  0], # 3
    [-1,  0,  0,  0,  0,  0], # 4
    [ 0,  0, -1,  0,  0,  0], # 5
    [ 0, -1,  0,  0,  0,  0], # 6
    [ 0,  0,  0, -1,  0,  0], # 7
    [ 0,  0,  0,  0,  0, -1], # 8
    [ 0,  0,  0,  0, -1,  0], # 9
    [ 0,  0,  0, -1,  1,  0], # 10
    [ 0, -1,  0,  0,  0,  1], # 11
    [ 0,  0,  0,  1, -1,  0], # 12
    [ 0,  1,  0,  0,  0, -1], # 13
    ], dtype=int)

## Basic Form of Gentic Toggle Switch

In [6]:
# Parameter values
kmu = 0.5
kmo = 0.00001
kp = 0.167
gamma_m = 0.05776
gamma_p = 0.00155
kr = 1.0
ku1 = 5.0
ku2 = 0.05

toggle_switch_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

In [7]:
# State with 1000 copies of everything, nothing bound to operators
toggle_switch_pop_0 = np.array([0, 1000, 0, 0, 0, 0], dtype=int)

toggle_switch_time_points = np.linspace(0, 80000, 4001)

In [8]:
# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(toggle_switch_propensity, 
                                toggle_switch_update, 
                                toggle_switch_pop_0, 
                                toggle_switch_time_points, 
                                args=toggle_switch_args)


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')

label_vector = ["gene 1", "gene 2"]

for c, i in enumerate([1, 3]):
    p.line(toggle_switch_time_points/60, 
           pop[0,:,i], color=colors[c], legend_label = label_vector[c])

bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

export_png(p, filename="../Results/Simulations/Toggle_Basic_Form.png")

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Toggle_Basic_Form.png'

## Varying levels of leakage

Each subplot features five realisations of the Gillespie algorithm for varying levels of kmo: (A) kmo = 0.00001, (B) kmo = 0.0001, (C) kmo = 0.001, (D) kmo = 0.005, (E) kmo = 0.05, (F) kmo = 0.05.

In [16]:
# Parameter values
kmu = 0.5
kmo = 0.00001
kp = 0.167
gamma_m = 0.05776
gamma_p = 0.00155
kr = 1.0
ku1 = 5.0
ku2 = 0.05

# State with 1000 copies of everything, nothing bound to operators
toggle_switch_pop_0 = np.array([0, 1000, 0, 0, 0, 0], dtype=int)

In [17]:
kmo = 0.00001

toggle_switch_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

no_of_runs  = 5

# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(toggle_switch_propensity, 
                                toggle_switch_update, 
                                toggle_switch_pop_0, 
                                toggle_switch_time_points, 
                                args=toggle_switch_args,
                                size = no_of_runs)


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')


for jj in range(no_of_runs):
    for c, i in enumerate([1, 3]):
        p.line(toggle_switch_time_points/60, 
               pop[jj,:,i], color=colors[c],
               line_width=0.8,alpha=0.2)
    
bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

png_name = "../Results/Simulations/Toggle_Leakage_" + str(kmo) + ".png"
export_png(p, filename = png_name)

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Toggle_Leakage_1e-05.png'

In [18]:
kmo = 0.0001

toggle_switch_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

no_of_runs  = 5

# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(toggle_switch_propensity, 
                                toggle_switch_update, 
                                toggle_switch_pop_0, 
                                toggle_switch_time_points, 
                                args=toggle_switch_args,
                                size = no_of_runs)


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')


for jj in range(no_of_runs):
    for c, i in enumerate([1, 3]):
        p.line(toggle_switch_time_points/60, 
               pop[jj,:,i], color=colors[c],
               line_width=0.8,alpha=0.2)
    
bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

png_name = "../Results/Simulations/Toggle_Leakage_" + str(kmo) + ".png"
export_png(p, filename = png_name)

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Toggle_Leakage_0.0001.png'

In [19]:
kmo = 0.001

toggle_switch_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

no_of_runs  = 5

# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(toggle_switch_propensity, 
                                toggle_switch_update, 
                                toggle_switch_pop_0, 
                                toggle_switch_time_points, 
                                args=toggle_switch_args,
                                size = no_of_runs)


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')


for jj in range(no_of_runs):
    for c, i in enumerate([1, 3]):
        p.line(toggle_switch_time_points/60, 
               pop[jj,:,i], color=colors[c],
               line_width=0.8,alpha=0.2)
    
bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

png_name = "../Results/Simulations/Toggle_Leakage_" + str(kmo) + ".png"
export_png(p, filename = png_name)

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Toggle_Leakage_0.001.png'

In [20]:
kmo = 0.005

toggle_switch_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

no_of_runs  = 5

# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(toggle_switch_propensity, 
                                toggle_switch_update, 
                                toggle_switch_pop_0, 
                                toggle_switch_time_points, 
                                args=toggle_switch_args,
                                size = no_of_runs)


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')


for jj in range(no_of_runs):
    for c, i in enumerate([1, 3]):
        p.line(toggle_switch_time_points/60, 
               pop[jj,:,i], color=colors[c],
               line_width=0.8,alpha=0.2)
    
bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

png_name = "../Results/Simulations/Toggle_Leakage_" + str(kmo) + ".png"
export_png(p, filename = png_name)

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Toggle_Leakage_0.005.png'

In [21]:
kmo = 0.01

toggle_switch_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

no_of_runs  = 5

# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(toggle_switch_propensity, 
                                toggle_switch_update, 
                                toggle_switch_pop_0, 
                                toggle_switch_time_points, 
                                args=toggle_switch_args,
                                size = no_of_runs)


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')


for jj in range(no_of_runs):
    for c, i in enumerate([1, 3]):
        p.line(toggle_switch_time_points/60, 
               pop[jj,:,i], color=colors[c],
               line_width=0.8,alpha=0.2)
    
bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

png_name = "../Results/Simulations/Toggle_Leakage_" + str(kmo) + ".png"
export_png(p, filename = png_name)

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Toggle_Leakage_0.01.png'

In [22]:
kmo = 0.05

toggle_switch_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

no_of_runs  = 5

# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(toggle_switch_propensity, 
                                toggle_switch_update, 
                                toggle_switch_pop_0, 
                                toggle_switch_time_points, 
                                args=toggle_switch_args,
                                size = no_of_runs)


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')


for jj in range(no_of_runs):
    for c, i in enumerate([1, 3]):
        p.line(toggle_switch_time_points/60, 
               pop[jj,:,i], color=colors[c],
               line_width=0.8,alpha=0.2)
    
bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

png_name = "../Results/Simulations/Toggle_Leakage_" + str(kmo) + ".png"
export_png(p, filename = png_name)

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Toggle_Leakage_0.05.png'